In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    master('local').\
    appName('recommender-system').\
    getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
df = spark.read.csv(
    path='D:/learn-ab/learning-PySpark/sample-data/movielens-ratings.csv',
    inferSchema=True,
    header=True
)
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [4]:
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [5]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
train_data, test_data = df.randomSplit([0.8, 0.2])

In [7]:
?ALS

Init signature:
ALS(
    *,
    rank: int = 10,
    maxIter: int = 10,
    regParam: float = 0.1,
    numUserBlocks: int = 10,
    numItemBlocks: int = 10,
    implicitPrefs: bool = False,
    alpha: float = 1.0,
    userCol: str = 'user',
    itemCol: str = 'item',
    seed: Optional[int] = None,
    ratingCol: str = 'rating',
    nonnegative: bool = False,
    checkpointInterval: int = 10,
    intermediateStorageLevel: str = 'MEMORY_AND_DISK',
    finalStorageLevel: str = 'MEMORY_AND_DISK',
    coldStartStrategy: str = 'nan',
    blockSize: int = 4096,
)
Docstring:     
Alternating Least Squares (ALS) matrix factorization.

ALS attempts to estimate the ratings matrix `R` as the product of
two lower-rank matrices, `X` and `Y`, i.e. `X * Yt = R`. Typically
these approximations are called 'factor' matrices. The general
approach is iterative. During each iteration, one of the factor
matrices is held constant, while the other is solved for using least
squares. The newly-solved factor matr

In [8]:
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating'
)

In [9]:
als_model = als.fit(train_data)
als_preds = als_model.transform(test_data)

In [10]:
als_preds.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      0|   3.0|    28|0.40441638|
|      3|   1.0|    26|  0.955033|
|      4|   1.0|    12| 2.6503055|
|      3|   1.0|    13| 1.2486162|
|      4|   2.0|    13| 2.3501046|
|      5|   1.0|    13| 1.0728198|
|      1|   1.0|     6| 1.4384822|
|      6|   1.0|     6|0.73272073|
|      1|   1.0|    20| 1.5648066|
|      4|   2.0|    20| 2.0827405|
|      1|   1.0|     5|   4.81714|
|      1|   1.0|    19| 0.7521463|
|      2|   1.0|    17| 0.8236364|
|      2|   4.0|     8|  4.024686|
|      2|   1.0|    23|  2.034547|
|      2|   2.0|     7| 2.3021975|
|      4|   1.0|     7| 0.6697476|
|      0|   1.0|    11| 1.5916053|
|      1|   1.0|    14| 1.1045423|
|      3|   1.0|     0|0.41607547|
+-------+------+------+----------+
only showing top 20 rows



In [11]:
als_eval = RegressionEvaluator(
    metricName='rmse',
    labelCol='rating',
    predictionCol='prediction'
)

In [12]:
rmse = als_eval.evaluate(als_preds)
print(f'RMSE for Recommendation System : {rmse}')

RMSE for Recommendation System : 1.760955972982973


In [13]:
single_user = test_data.filter(test_data['userId']==11).select('movieId', 'userId')
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      9|    11|
|     20|    11|
|     22|    11|
|     23|    11|
|     25|    11|
|     36|    11|
|     37|    11|
|     40|    11|
|     61|    11|
|     62|    11|
|     67|    11|
|     97|    11|
+-------+------+



In [14]:
recommendations = als_model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     23|    11|  4.6228976|
|      9|    11|   4.439014|
|     25|    11|  3.4724045|
|     37|    11|  3.4162583|
|     36|    11|  2.7046664|
|     62|    11|  2.6601183|
|     22|    11|  2.5334475|
|     61|    11|  2.2546299|
|      0|    11|  1.5916053|
|     97|    11|  1.4051706|
|     20|    11|  1.3864875|
|     67|    11|  1.1807175|
|     40|    11|0.026035845|
+-------+------+-----------+

